## 순환신경망 구현 및 학습

In [1]:
import tensorflow as tf

## 하이퍼 파라미터

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## 모델 정의

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.rnn = tf.keras.layers.SimpleRNN(32)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.rnn(x)
        return self.dense(x)

## 학습, 테스트 루프 정의

In [4]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
    predictions = model(inputs, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [5]:
imdb = tf.keras.datasets.imdb

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=32)

X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                       value=0,
                                                      padding='pre',
                                                      maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [6]:
# Create model
model = MyModel()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [7]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5460605025291443, Accuracy: 70.34400177001953, Test Loss: 0.4561103880405426, Test Accuracy: 78.49199676513672
Epoch 2, Loss: 0.3589100241661072, Accuracy: 84.32400512695312, Test Loss: 0.4808015525341034, Test Accuracy: 78.36400604248047
Epoch 3, Loss: 0.21455258131027222, Accuracy: 91.70800018310547, Test Loss: 0.6066750288009644, Test Accuracy: 76.61599731445312
Epoch 4, Loss: 0.09188593924045563, Accuracy: 96.96800231933594, Test Loss: 0.8169109225273132, Test Accuracy: 74.35600280761719
Epoch 5, Loss: 0.03929673135280609, Accuracy: 98.7239990234375, Test Loss: 1.021120548248291, Test Accuracy: 73.79999542236328
Epoch 6, Loss: 0.01827871985733509, Accuracy: 99.49200439453125, Test Loss: 1.219519853591919, Test Accuracy: 75.70000457763672
Epoch 7, Loss: 0.01426525879651308, Accuracy: 99.59200286865234, Test Loss: 1.2706654071807861, Test Accuracy: 73.88800048828125
Epoch 8, Loss: 0.02123543992638588, Accuracy: 99.31200408935547, Test Loss: 1.3168946504592896, Test A

## Implementing with LSTM

In [8]:
class MyModel_lstm(tf.keras.Model):
    def __init__(self):
        super(MyModel_lstm, self).__init__()
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)
        self.lstm = tf.keras.layers.LSTM(32)
        self.dense = tf.keras.layers.Dense(2, activation='softmax')
        
    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.lstm(x)
        return self.dense(x)

In [9]:
# Create model
model = MyModel_lstm()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [10]:
# Implement training loop
@tf.function
def train_step(model, inputs, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, inputs, labels, loss_object, test_loss, test_accuracy):
    predictions = model(inputs, training=False)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
for epoch in range(EPOCHS):
    for seqs, labels in train_ds:
        train_step(model, seqs, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_seqs, test_labels in test_ds:
        test_step(model, test_seqs, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.5032906532287598, Accuracy: 74.04399871826172, Test Loss: 0.4356394112110138, Test Accuracy: 79.44800567626953
Epoch 2, Loss: 0.35902199149131775, Accuracy: 84.10400390625, Test Loss: 0.45492494106292725, Test Accuracy: 78.9280014038086
Epoch 3, Loss: 0.2875344157218933, Accuracy: 87.85200500488281, Test Loss: 0.47499218583106995, Test Accuracy: 77.48400115966797
Epoch 4, Loss: 0.22897759079933167, Accuracy: 90.85199737548828, Test Loss: 0.549153745174408, Test Accuracy: 76.97200012207031
Epoch 5, Loss: 0.18192505836486816, Accuracy: 93.04000091552734, Test Loss: 0.6949563026428223, Test Accuracy: 75.77999877929688
Epoch 6, Loss: 0.14181070029735565, Accuracy: 94.72000122070312, Test Loss: 0.8265852928161621, Test Accuracy: 75.14799499511719
Epoch 7, Loss: 0.10652672499418259, Accuracy: 96.14800262451172, Test Loss: 0.9874829649925232, Test Accuracy: 75.1240005493164
Epoch 8, Loss: 0.07797785848379135, Accuracy: 97.23600006103516, Test Loss: 1.1331263780593872, Test Ac